In [1]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as python_random

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import regularizers
import pickle

2023-03-13 22:45:25.709774: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
aggregated_data = pickle.load( open( '../data/processed/20230304/aggregated_data', 'rb' ) )

In [3]:
[train_mps_raw_pca, valid_mps_raw_pca, test_mps_raw_pca,
    train_mps_bg_pca, valid_mps_bg_pca, test_mps_bg_pca,
    train_mps_fg_pca, valid_mps_fg_pca, test_mps_fg_pca,
    train_indices_raw_pca, valid_indices_raw_pca, test_indices_raw_pca,
    train_indices_bg_pca, valid_indices_bg_pca, test_indices_bg_pca,
    train_indices_fg_pca, valid_indices_fg_pca, test_indices_fg_pca,
    train_embedding_raw_pca, valid_embedding_raw_pca, test_embedding_raw_pca,
    train_embedding_bg_pca, valid_embedding_bg_pca, test_embedding_bg_pca,
    train_embedding_fg_pca, valid_embedding_fg_pca, test_embedding_fg_pca,
    train_vgg_raw_pca, valid_vgg_raw_pca, test_vgg_raw_pca,
    train_vgg_bg_pca, valid_vgg_bg_pca, test_vgg_bg_pca,
    train_vgg_fg_pca, valid_vgg_fg_pca, test_vgg_fg_pca,
    train_panns_clip_raw_pca, valid_panns_clip_raw_pca, test_panns_clip_raw_pca,
    train_panns_clip_bg_pca, valid_panns_clip_bg_pca, test_panns_clip_bg_pca,
    train_panns_clip_fg_pca, valid_panns_clip_fg_pca, test_panns_clip_fg_pca,
    train_panns_embedding_raw_pca, valid_panns_embedding_raw_pca, test_panns_embedding_raw_pca,
    train_panns_embedding_bg_pca, valid_panns_embedding_bg_pca, test_panns_embedding_bg_pca,
    train_panns_embedding_fg_pca, valid_panns_embedding_fg_pca, test_panns_embedding_fg_pca,
    y_train, y_valid, y_test] = aggregated_data

In [4]:
# YAMNet
train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_embedding_raw_pca, train_vgg_raw_pca), axis = 1)
valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_embedding_raw_pca, valid_vgg_raw_pca), axis = 1)
test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_embedding_raw_pca, test_vgg_raw_pca), axis = 1)

train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_embedding_bg_pca, train_vgg_bg_pca), axis = 1)
valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_embedding_bg_pca, valid_vgg_bg_pca), axis = 1)
test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_embedding_bg_pca, test_vgg_bg_pca), axis = 1)

train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_embedding_fg_pca, train_vgg_fg_pca), axis = 1)
valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_embedding_fg_pca, valid_vgg_fg_pca), axis = 1)
test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_embedding_fg_pca, test_vgg_fg_pca), axis = 1)

train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)

In [5]:
# # VGG
# train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_vgg_raw_pca), axis = 1)
# valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_vgg_raw_pca), axis = 1)
# test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_vgg_raw_pca), axis = 1)

# train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_vgg_bg_pca), axis = 1)
# valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_vgg_bg_pca), axis = 1)
# test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_vgg_bg_pca), axis = 1)

# train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_vgg_fg_pca), axis = 1)
# valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_vgg_fg_pca), axis = 1)
# test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_vgg_fg_pca), axis = 1)

# train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
# valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
# test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)

In [6]:
# # including panns embeddings
# train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_panns_embedding_raw_pca), axis = 1)
# valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_panns_embedding_raw_pca), axis = 1)
# test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_panns_embedding_raw_pca), axis = 1)

# train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_panns_embedding_bg_pca), axis = 1)
# valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_panns_embedding_bg_pca), axis = 1)
# test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_panns_embedding_bg_pca), axis = 1)

# train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_panns_embedding_fg_pca), axis = 1)
# valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_panns_embedding_fg_pca), axis = 1)
# test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_panns_embedding_fg_pca), axis = 1)

# train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
# valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
# test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)

In [7]:
train_data_fg.shape

(11933, 558)

In [8]:
train_data_all.shape

(11933, 1674)

# Lazy predict

In [9]:
# from lazypredict.Supervised import LazyClassifier
# clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(train_vgg_bg_pca, valid_vgg_bg_pca, y_train, y_valid)
# models

# Tensorflow parameter tuning

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt


In [11]:
def build_model_narrow(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_raw.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.2, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 10)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=10, max_value=200, step=10),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(2))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryFocalCrossentropy(gamma=2.0, from_logits=True),
#         loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model

def build_model_wide(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_all.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.2, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 10)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=10, max_value=500, step=50),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(2))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model

In [12]:
def build_model_AUC_narrow(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_raw.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.2, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=10, max_value=200, step=10),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#         loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.AUC(name='AUC')],
    )

    return model

def build_model_AUC_wide(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_all.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.2, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=10, max_value=500, step=50),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.AUC(name='AUC')],
    )

    return model

In [13]:
directory = '../model/BinaryCrossentropy_AUC'
feat_name = '0313_YAMNetEmbedding'

tuner_raw = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory=directory,
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_raw_'+feat_name)

tuner_bg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory=directory,
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_bg_'+feat_name)

tuner_fg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory=directory,
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_fg_'+feat_name)

tuner_all = kt.BayesianOptimization(build_model_AUC_wide,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory=directory,
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_all_'+feat_name)

2023-03-13 22:45:31.698329: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [15]:
tuner_all.search(train_data_all, y_train, epochs=50, validation_data=(valid_data_all, y_valid), callbacks=[stop_early], use_multiprocessing=True)

Trial 100 Complete [00h 00m 27s]
AUC: 0.9128738045692444

Best AUC So Far: 0.9748368859291077
Total elapsed time: 01h 09m 17s
INFO:tensorflow:Oracle triggered exit


In [16]:
tuner_bg.search(train_data_bg, y_train, epochs=50, validation_data=(valid_data_bg, y_valid), callbacks=[stop_early], use_multiprocessing=True)

Trial 100 Complete [00h 00m 15s]
AUC: 0.7725697755813599

Best AUC So Far: 0.9170337319374084
Total elapsed time: 00h 25m 31s
INFO:tensorflow:Oracle triggered exit


In [17]:
tuner_raw.search(train_data_raw, y_train, epochs=50, validation_data=(valid_data_raw, y_valid), callbacks=[stop_early], use_multiprocessing=True)

Trial 100 Complete [00h 00m 06s]
AUC: 0.7988672852516174

Best AUC So Far: 0.9164780974388123
Total elapsed time: 00h 21m 51s
INFO:tensorflow:Oracle triggered exit


In [18]:
tuner_fg.search(train_data_fg, y_train, epochs=50, validation_data=(valid_data_fg, y_valid), callbacks=[stop_early], use_multiprocessing=True)

Trial 100 Complete [00h 00m 09s]
AUC: 0.9202155470848083

Best AUC So Far: 0.9202155470848083
Total elapsed time: 00h 16m 21s
INFO:tensorflow:Oracle triggered exit


In [19]:
tuner_raw.results_summary(1)

Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_raw_0313_YAMNetEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 200
activation: relu
lr: 0.0006924712314276848
units_1: 200
units_2: 30
units_3: 10
units_4: 200
Score: 0.9164780974388123


In [20]:
tuner_bg.results_summary(1)

Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_bg_0313_YAMNetEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 200
activation: relu
lr: 0.0008608506839418887
units_1: 60
units_2: 70
units_3: 140
units_4: 10
Score: 0.9170337319374084


In [21]:
tuner_fg.results_summary(1)

Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_fg_0313_YAMNetEmbedding
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 200
activation: relu
lr: 0.0005887145125682295
units_1: 10
units_2: 80
units_3: 100
units_4: 170
Score: 0.9202155470848083


In [27]:
tuner_all.results_summary(10)

Results summary
Results in ../model/BinaryCrossentropy_AUC/kt_Bayesian_all_0313_YAMNetEmbedding
Showing 10 best trials
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 460
activation: relu
lr: 0.000323865228707218
units_1: 460
units_2: 210
units_3: 210
units_4: 110
Score: 0.9748368859291077
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 460
activation: relu
lr: 0.0004294955345256393
units_1: 310
units_2: 310
units_3: 210
units_4: 460
Score: 0.9742940664291382
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 460
activation: relu
lr: 0.0004102512283488621
units_1: 460
units_2: 110
units_3: 310
units_4: 60
Score: 0.9741812944412231
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 460
activation: relu
lr: 0.0003133717680926488
units_1: 210
units_2: 260
units_3: 210
units_4: 460
Score: 0.9739019870758057
Trial summary
Hyperparameters:
dr: 0.2
num_layers: 1
units_0: 460
activation: relu
lr: 0.00036641162505939517
units_1: 460
units_2: 260

In [26]:
tuner_all.get_best_hyperparameters(num_trials=1)[0]